# Descripción del problema


Se desea realizar un experimento para comparar modelos con los que detectar el fraude bancario. El conjunto de datos fue creado por Sérgio Jesus, José Pombal, Duarte Alves, André F. Cruz, Pedro Saleiro y Pedro Bizarro en nombre de Feedzai. 

Se procederá al tratamiento de los datos, modelado general, optimizacion de modelos con mejores resultados de origen y estudio de la explicabilidad de dicho modelo.

# Descripción del dataset

Cada instancia en la serie de conjuntos de datos representa una solicitud sintética de apertura de cuenta bancaria con ingeniería de características en formato tabular. Estas fueron generadas utilizando un CTGAN entrenado en un  conjunto de datos anonimizado del mundo real sobre fraudes en la apertura de cuentas bancarias. No existen diferentes tipos de instancias.


Cada instancia es una solicitud de cuenta bancaria sintética con características diseñadas que incluyen los siguientes campos:
1. **income (numeric)**: Annual income of the applicant (in decile form). Ranges between [0.1, 0.9].
1. **name_email_similarity (numeric)**: Metric of similarity between email and applicant’s name. Higher values represent higher similarity. Ranges between [0, 1].
1. **prev_address_months_count (numeric)**: Number of months in previous registered address of the applicant, i.e. the applicant’s previous residence, if applicable. Ranges between [−1, 380] months (-1 is a missing value).
1. **current_address_months_count (numeric)**: Months in currently registered address of the applicant. Ranges between [−1, 429] months (-1 is a missing value).
1. **customer_age (numeric)**: Applicant’s age in years, rounded to the decade. Ranges between [10, 90] years.
1. **days_since_request (numeric)**: Number of days passed since application was done. Ranges between [0, 79] days.
1. **intended_balcon_amount (numeric)**: Initial transferred amount for application. Ranges between [−16, 114] (negatives are missing values).
1. **payment_type (categorical)**: Credit payment plan type. 5 possible (annonymized) values.
1. **zip_count_4w (numeric)**: Number of applications within same zip code in last 4 weeks. Ranges between [1, 6830].
1. **velocity_6h (numeric)**: Velocity of total applications made in last 6 hours i.e., average number of applications per hour in the last 6 hours. Ranges between [−175, 16818].
1. **velocity_24h (numeric)**: Velocity of total applications made in last 24 hours i.e., average number of applications per hour in the last 24 hours. Ranges between [1297, 9586]
1. **velocity_4w (numeric)**: Velocity of total applications made in last 4 weeks, i.e., average number of applications per hour in the last 4 weeks. Ranges between [2825, 7020].
1. **bank_branch_count_8w (numeric)**: Number of total applications in the selected bank branch in last 8 weeks. Ranges between [0, 2404].
1. **date_of_birth_distinct_emails_4w (numeric)**: Number of emails for applicants with same date of birth in last 4 weeks. Ranges between [0, 39].
1. **employment_status (categorical)**: Employment status of the applicant. 7 possible (annonymized) values.
1. **credit_risk_score (numeric)**: Internal score of application risk. Ranges between [−191, 389].
1. **email_is_free (binary)**: Domain of application email (either free or paid).
1. **housing_status (categorical)**: Current residential status for applicant. 7 possible (annonymized) values.
1. **phone_home_valid (binary)**: Validity of provided home phone.
1. **phone_mobile_valid (binary)**: Validity of provided mobile phone.
1. **bank_months_count (numeric)**: How old is previous account (if held) in months.Ranges between [−1, 32] months (-1 is a missing value).
1. **has_list_num_vars_cards (binary)**: If applicant has list_num_vars cards from the same banking company.
1. **proposed_credit_limit (numeric)**: Applicant’s proposed credit limit. Ranges between [200, 2000].
1. **foreign_request (binary)**: If origin country of request is different from bank’s country.
1. **source (categorical)**: Online source of application. Either browser (INTERNET) or app (TELEAPP).
1. **session_length_in_minutes (numeric)**: Length of user session in banking website in minutes. Ranges between [−1, 107] minutes (-1 is a missing value).
1. **device_os (categorical)**: Operative system of device that made request. Possible values are: Windows, macOS, Linux, X11, or list_num_vars.
1. **keep_alive_session (binary)**: User option on session logout.
1. **device_distinct_emails (numeric)**: Number of distinct emails in banking website from the used device in last 8 weeks. Ranges between [−1, 2] emails (-1 is a missing value).
1. **device_fraud_count (numeric)**: Number of fraudulent applications with used device. Ranges between [0, 1].
1. **month (numeric)**: Month where the application was made. Ranges between [0, 7].
1. **fraud_bool (binary)**: If the application is fraudulent or not



La estructura del proyecto es:

1. 	Diccionario de datos, planteamiento del problema Preprocesamiento (notebook 1)
2.	Análisis exploratorio de datos EDA (notebook 2)
3.	Tratamiento de los datos, Selección e ingeniería de variables (notebook 3)
4.	Estructuración del modelado y modelo base y selección de métrica (notebook 4)
5.	Modelado inicial (notebook 5 - 13)
6.	Optimización de hiperparámetros de los seleccionados (usando val)
7.	Comparación de modelos (usando test) Explicabilidad y Conclusiones (notebook 14)


## Importo librerias


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

In [2]:
def dame_variables_categoricas(dataset=None):
    '''
    ----------------------------------------------------------------------------------------------------------
    Función dame_variables_categoricas:
    ----------------------------------------------------------------------------------------------------------
        -Descripción: Función que recibe un dataset y devuelve una lista con los nombres de las 
        variables categóricas
        -Inputs: 
            -- dataset: Pandas dataframe que contiene los datos
        -Return:
            -- lista_variables_categoricas: lista con los nombres de las variables categóricas del
            dataset de entrada con menos de 100 valores diferentes
            -- 1: la ejecución es incorrecta
    '''
    if dataset is None:
        print(u'\nFaltan argumentos por pasar a la función')
        return 1
    lista_variables_categoricas = []
    list_num_vars = []
    for i in dataset.columns:
        if (dataset[i].dtype!=float) & (dataset[i].dtype!=int):
            unicos = int(len(np.unique(dataset[i].dropna(axis=0, how='all'))))
            if unicos < 100:
                lista_variables_categoricas.append(i)
            else:
                list_num_vars.append(i)

    return lista_variables_categoricas, list_num_vars

## Leo los datos

In [3]:
path_folder = "../data/"
pd_BAF = pd.read_csv(path_folder +"Base.csv")

Analisis generales de la tabla


Dimensión



In [4]:
print(pd_BAF.shape, pd_BAF.drop_duplicates().shape)


(1000000, 32) (1000000, 32)


Tipos de datos

In [5]:
pd_BAF.dtypes.to_dict()

{'fraud_bool': dtype('int64'),
 'income': dtype('float64'),
 'name_email_similarity': dtype('float64'),
 'prev_address_months_count': dtype('int64'),
 'current_address_months_count': dtype('int64'),
 'customer_age': dtype('int64'),
 'days_since_request': dtype('float64'),
 'intended_balcon_amount': dtype('float64'),
 'payment_type': dtype('O'),
 'zip_count_4w': dtype('int64'),
 'velocity_6h': dtype('float64'),
 'velocity_24h': dtype('float64'),
 'velocity_4w': dtype('float64'),
 'bank_branch_count_8w': dtype('int64'),
 'date_of_birth_distinct_emails_4w': dtype('int64'),
 'employment_status': dtype('O'),
 'credit_risk_score': dtype('int64'),
 'email_is_free': dtype('int64'),
 'housing_status': dtype('O'),
 'phone_home_valid': dtype('int64'),
 'phone_mobile_valid': dtype('int64'),
 'bank_months_count': dtype('int64'),
 'has_other_cards': dtype('int64'),
 'proposed_credit_limit': dtype('float64'),
 'foreign_request': dtype('int64'),
 'source': dtype('O'),
 'session_length_in_minutes': dty

## Exploración de la variable objetivo y tratamiento inicial

In [6]:
pd_plot_fraud_bool = round(pd_BAF['fraud_bool']\
        .value_counts(normalize=True)\
        .mul(100).rename('percent').reset_index(), 2)

pd_plot_fraud_bool_conteo = pd_BAF['fraud_bool'].value_counts().reset_index()
pd_plot_fraud_bool_pc = pd.merge(pd_plot_fraud_bool, pd_plot_fraud_bool_conteo, how='inner')
pd_plot_fraud_bool_pc


,fraud_bool,percent,count
0,0,98.9,988971
1,1,1.1,11029


In [7]:
fig = px.histogram(pd_plot_fraud_bool_pc, x="fraud_bool", y=['percent'])
fig.show()

Selección de threshold por filas y columnas para eliminar valores missing

In [8]:
pd_series_null_columns = pd_BAF.isnull().sum().sort_values(ascending=False)
pd_series_null_rows = pd_BAF.isnull().sum(axis=1).sort_values(ascending=False)
print(pd_series_null_columns.shape, pd_series_null_rows.shape)

pd_null_columnas = pd.DataFrame(pd_series_null_columns, columns=['nulos_columnas'])     
pd_null_filas = pd.DataFrame(pd_series_null_rows, columns=['nulos_filas'])  
pd_null_filas['target'] = pd_BAF['fraud_bool'].copy()
pd_null_columnas['porcentaje_columnas'] = pd_null_columnas['nulos_columnas']/pd_BAF.shape[0]
pd_null_filas['porcentaje_filas']= pd_null_filas['nulos_filas']/pd_BAF.shape[1]

(32,) (1000000,)


In [9]:
pd_BAF.shape

(1000000, 32)

In [10]:
pd_null_columnas

,nulos_columnas,porcentaje_columnas
fraud_bool,0,0.0
income,0,0.0
device_fraud_count,0,0.0
device_distinct_emails_8w,0,0.0
keep_alive_session,0,0.0
device_os,0,0.0
session_length_in_minutes,0,0.0
source,0,0.0
foreign_request,0,0.0
proposed_credit_limit,0,0.0


In [11]:
threshold=0.9
list_vars_not_null = list(pd_null_columnas[pd_null_columnas['porcentaje_columnas']<threshold].index)
pd_BAF_filter_null = pd_BAF.loc[:, list_vars_not_null]
pd_BAF_filter_null.shape

(1000000, 32)

In [12]:
pd_null_filas

,nulos_filas,target,porcentaje_filas
0,0,1,0.0
666657,0,0,0.0
666659,0,0,0.0
666660,0,0,0.0
666661,0,0,0.0
...,...,...,...
333337,0,0,0.0
333338,0,0,0.0
333339,0,0,0.0
333340,0,0,0.0


## Tratamiento valores missing

In [13]:
variables = ['prev_address_months_count', 'current_address_months_count','intended_balcon_amount', 'bank_months_count', 'session_length_in_minutes', 'device_distinct_emails_8w']
missing_count = []
missing_percentage = []

for variable in variables:
    if variable == 'intended_balcon_amount':
        missing_count.append(pd_BAF[pd_BAF[variable] < 0].shape[0])
    else:
        missing_count.append(pd_BAF[pd_BAF[variable] == -1].shape[0])

    missing_percentage.append((missing_count[-1] / len(pd_BAF)) * 100)

missing_data_df = pd.DataFrame({
    'Variable': variables,
    'Missing': missing_count,
    'Porcentaje Missing': missing_percentage
})

missing_data_df

,Variable,Missing,Porcentaje Missing
0,prev_address_months_count,712920,71.2920
1,current_address_months_count,4254,0.4254
2,intended_balcon_amount,742523,74.2523
3,bank_months_count,253635,25.3635
4,session_length_in_minutes,2015,0.2015
5,device_distinct_emails_8w,359,0.0359


Se puede notar la presencia de valores nulos en estas cinco variables; sin embargo, dado que se han gestionado previamente, no impactan en el análisis de nuestro conjunto de datos. 
Las variables 'prev_address_months_count' y 'intended_balcon_amount' son las que contienen la mayor cantidad de estos valores. Dada la naturaleza de estas no se requiere ninguna operacion

## Tipos: Variables categoricas y numericas

In [14]:
list_cat_vars, list_num_vars = dame_variables_categoricas(dataset=pd_BAF_filter_null)
pd_BAF_filter_null[list_cat_vars] = pd_BAF_filter_null[list_cat_vars].astype("category")
pd_BAF_filter_null[list_cat_vars].head()

,fraud_bool,device_fraud_count,device_distinct_emails_8w,keep_alive_session,device_os,source,foreign_request,has_other_cards,bank_months_count,phone_mobile_valid,phone_home_valid,housing_status,email_is_free,employment_status,date_of_birth_distinct_emails_4w,payment_type,customer_age,month
0,1,0,1,0,windows,INTERNET,0,0,24,0,1,BA,0,CA,6,AA,50,7
1,1,0,1,0,windows,INTERNET,0,0,15,0,0,BA,1,CA,3,AB,50,7
2,1,0,1,0,other,INTERNET,0,0,-1,1,0,BA,1,CB,14,AC,40,7
3,1,0,1,0,linux,INTERNET,0,1,31,1,0,BA,1,CA,6,AB,50,7
4,1,0,1,1,macintosh,INTERNET,0,0,31,0,1,BA,1,CA,2,AB,50,7


In [15]:
pd_BAF_filter_null[list_cat_vars].dtypes

fraud_bool                          category
device_fraud_count                  category
device_distinct_emails_8w           category
keep_alive_session                  category
device_os                           category
source                              category
foreign_request                     category
has_other_cards                     category
bank_months_count                   category
phone_mobile_valid                  category
phone_home_valid                    category
housing_status                      category
email_is_free                       category
employment_status                   category
date_of_birth_distinct_emails_4w    category
payment_type                        category
customer_age                        category
month                               category
dtype: object

In [16]:
pd_BAF_filter_null[list_cat_vars].head(5)

,fraud_bool,device_fraud_count,device_distinct_emails_8w,keep_alive_session,device_os,source,foreign_request,has_other_cards,bank_months_count,phone_mobile_valid,phone_home_valid,housing_status,email_is_free,employment_status,date_of_birth_distinct_emails_4w,payment_type,customer_age,month
0,1,0,1,0,windows,INTERNET,0,0,24,0,1,BA,0,CA,6,AA,50,7
1,1,0,1,0,windows,INTERNET,0,0,15,0,0,BA,1,CA,3,AB,50,7
2,1,0,1,0,other,INTERNET,0,0,-1,1,0,BA,1,CB,14,AC,40,7
3,1,0,1,0,linux,INTERNET,0,1,31,1,0,BA,1,CA,6,AB,50,7
4,1,0,1,1,macintosh,INTERNET,0,0,31,0,1,BA,1,CA,2,AB,50,7


In [17]:
print(pd_BAF_filter_null["customer_age"].value_counts().count())
pd_BAF_filter_null["customer_age"]\
    .apply(lambda x: str(x).lower().strip()).value_counts(normalize=True)#.count()

9


customer_age
30    0.311433
20    0.245855
40    0.238712
50    0.140353
60    0.034770
10    0.020987
70    0.006517
80    0.001297
90    0.000076
Name: proportion, dtype: float64

In [18]:
pd_BAF_filter_null.shape

(1000000, 32)

## Guardado de la tabla

In [20]:
pd_BAF_filter_null.to_csv(path_folder + "pd_data_initial_preprocessing.csv")